In [3]:
pip install tensorflow

     -------------------------------------- 276.5/276.5 MB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 5.6/5.6 MB 823.6 kB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 6.0 MB/s eta 0:00:00
     ---------------------------------------- 4.3/4.3 MB 3.9 MB/s eta 0:00:00
     -------------------------------------- 65.5/65.5 kB 876.9 kB/s eta 0:00:00
     ---------------------------------------- 24.4/24.4 MB 3.6 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 4.7 MB/s eta 0:00:00
     -------------------------------------- 126.5/126.5 kB 2.5 MB/s eta 0:00:00
     -------------------------------------- 440.8/440.8 kB 4.6 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB 3.0 MB/s eta 0:00:00
     -------------------------------------- 181.8/181.8 kB 3.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import tensorflow as tf
print(tf.__version__)

2.13.0


In [5]:
import numpy as np
import pandas as pd
import sys
import mlflow
import warnings
import pickle
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import ticker

from statsmodels.tsa.stattools import adfuller, acf, pacf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [6]:
%matplotlib inline
warnings.filterwarnings("ignore")

only the date and sales columns will be used for LSTM

In [7]:
train_sales = pd.read_csv('clean_train_store.csv', parse_dates = True, low_memory = False)

In [8]:
data = train_sales.groupby("Date").agg({"Sales": "mean"})
data

,Sales
Date,
2013-01-01,18.755556
2013-01-02,5991.964444
2013-01-03,5400.386667
2013-01-04,5716.217778
2013-01-05,5104.604444
...,...
2015-07-27,9330.057522
2015-07-28,7892.123894
2015-07-29,7428.000000


In [9]:
scaler = StandardScaler()
scaled_array = scaler.fit_transform(data)
data['DataScaled'] = scaled_array

In [10]:
SIZE = len(data.DataScaled)
WINDOW_SIZE = 48
BATCH_SIZE = SIZE - WINDOW_SIZE * 2
EPOCHS = 200

In [11]:
DateTrain = data.index.values[0:BATCH_SIZE]
DateValid = data.index.values[BATCH_SIZE:]
XTrain = data.DataScaled.values[0:BATCH_SIZE].astype('float32')
XValid = data.DataScaled.values[BATCH_SIZE:].astype('float32')

# Obtain shapes for vectors of size (,1) for dates series

DateTrain = np.reshape(DateTrain, (-1, 1))
DateValid = np.reshape(DateValid, (-1, 1))

print("Shape of the training set date series: ", DateTrain.shape)
print("Shape of the validation set date series: ", DateValid.shape)
print()
print("Shape of the training set logarithm of sales series: ", XTrain.shape)
print("Shape of the validation set logarithm of sales series in a stateless LSTM: ", XValid.shape)

Shape of the training set date series:  (846, 1)
Shape of the validation set date series:  (96, 1)

Shape of the training set logarithm of sales series:  (846,)
Shape of the validation set logarithm of sales series in a stateless LSTM:  (96,)


In [12]:
tf.random.set_seed(1234)
# add extra dimension
series = tf.expand_dims(XTrain, axis=-1)
series.shape

TensorShape([846, 1])

In [13]:
# create tensor from each individual element
dataset = tf.data.Dataset.from_tensor_slices(series)
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(1,), dtype=tf.float32, name=None)>

In [14]:
# takes a window_size + 1 chunk from the slices
dataset = dataset.window(WINDOW_SIZE + 1, shift=1, drop_remainder=True)

In [15]:
# Example of Window
datasetEx = tf.data.Dataset.from_tensor_slices(tf.range(10))
datasetEx = datasetEx.window(5, shift=1, drop_remainder=True)
for window in datasetEx:
    print([elem.numpy() for elem in window])

[0, 1, 2, 3, 4]
[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]
[3, 4, 5, 6, 7]
[4, 5, 6, 7, 8]
[5, 6, 7, 8, 9]


In [16]:
dataset = dataset.flat_map(lambda window: window.batch(WINDOW_SIZE + 1))

In [17]:
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))

In [18]:
dataset = dataset.batch(BATCH_SIZE).prefetch(1)

In [19]:
def windowed_dataset(series, window_size=WINDOW_SIZE, batch_size=BATCH_SIZE):
  series = tf.expand_dims(series, axis=-1)
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
  dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset

In [20]:
DatasetTrain = windowed_dataset(XTrain)
DatasetVal = windowed_dataset(XValid)

# Model Training

In [21]:
model = Sequential()
model.add(LSTM(8, input_shape=[None, 1], return_sequences=True))
model.add(LSTM(4, input_shape=[None, 1]))
model.add(Dense(1))
model.compile(loss="huber_loss", optimizer='adam')

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 8)           320       
                                                                 
 lstm_1 (LSTM)               (None, 4)                 208       
                                                                 
 dense (Dense)               (None, 1)                 5         
                                                                 
Total params: 533 (2.08 KB)
Trainable params: 533 (2.08 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
mlflow.set_experiment("LSTM")
mlflow.tensorflow.autolog()
history = model.fit(DatasetTrain, epochs=EPOCHS, validation_data=DatasetVal, verbose=1)

2023/09/02 18:12:37 INFO mlflow.tracking.fluent: Experiment with name 'LSTM' does not exist. Creating a new experiment.
2023/09/02 18:12:38 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '03a2d79c14aa4b6a93f86086da645a89', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/200
1/1 [==============================] - 28s 28s/step - loss: 0.4152 - val_loss: 0.4110
Epoch 2/200
1/1 [==============================] - 1s 662ms/step - loss: 0.4147 - val_loss: 0.4103
Epoch 3/200
1/1 [==============================] - 1s 635ms/step - loss: 0.4143 - val_loss: 0.4095
Epoch 4/200
1/1 [==============================] - 1s 607ms/step - loss: 0.4138 - val_loss: 0.4088
Epoch 5/200
1/1 [==============================] - 1s 667ms/step - loss: 0.4133 - val_loss: 0.4080
Epoch 6/200
1/1 [==============================] - 1s 647ms/step - loss: 0.4129 - val_loss: 0.4072
Epoch 7/200
1/1 [==============================] - 1s 659ms/step - loss: 0.4124 - val_loss: 0.4064
Epoch 8/200
1/1 [==============================] - 1s 593ms/step - loss: 0.4120 - val_loss: 0.4057
Epoch 9/200
1/1 [==============================] - 1s 600ms/step - loss: 0.4116 - val_loss: 0.4049
Epoch 10/200
1/1 [==============================] - 1s 641ms/step - loss: 0.4111 - val_loss: 0.4041
Epoch 11/2

INFO:tensorflow:Assets written to: C:\Users\Admin\AppData\Local\Temp\tmpwmonnz6q\model\data\model\assets


In [24]:
from time import gmtime, strftime

In [25]:
time = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model.save(f'../models/LSTM_sales-{time}.pkl')

INFO:tensorflow:Assets written to: ../models/LSTM_sales-2023-09-02-12-51-22.pkl\assets


INFO:tensorflow:Assets written to: ../models/LSTM_sales-2023-09-02-12-51-22.pkl\assets
